In [32]:
import pandas as pd
submissions = pd.read_csv("Raw_data\\sel_hn_stories.csv")
submissions.head(2)

,2014-06-24T05:50:40.000Z,1,flux7.com,8 Ways to Use Docker in the Real World
0,2010-02-17T16:57:59Z,1,blog.jonasbandi.net,Software: Sadly we did adopt from the construc...
1,2014-02-04T02:36:30Z,1,blogs.wsj.com,Google’s Stock Split Means More Control for L...


In [33]:
import pandas as pd
submissions = pd.read_csv("Raw_data\\sel_hn_stories.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()
submissions.head(2)

,submission_time,upvotes,url,headline
0,2010-02-17T16:57:59Z,1,blog.jonasbandi.net,Software: Sadly we did adopt from the construc...
1,2014-02-04T02:36:30Z,1,blogs.wsj.com,Google’s Stock Split Means More Control for L...


In [5]:
# convert each headline to a numerical representation using a bag of words model which represents each piece of text as a numerical vector by splitting each sentence into a list of individual words, or tokens. The split occurs on the space character (" ")

tokenized_headlines = []
for item in submissions["headline"]:
    tokenized_headlines.append(item.split())

In [6]:
# clean words/tokens

punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)

In [7]:
# converting the sentences to their numerical representations

import numpy as np
unique_tokens = []
single_tokens = []
for item in clean_tokenized:
    for token in item:
        if token not in unique_tokens:
            single_tokens.append(token)
        unique_tokens.append(token)
counts = pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)

In [8]:
# fill in the correct counts for each cell

for idx, item in enumerate(clean_tokenized):
    for token in item:
        if token in unique_tokens:
            counts.iloc[idx][token] += 1


In [14]:
#  remove any words that occur fewer than 5 times or more than 100 times to make better predictions

word_counts = counts.sum(axis =0)
counts = counts.loc[:,(word_counts >= 5) & (word_counts <= 100)]

In [15]:
counts.head(2)

,software,we,did,from,googles,means,more,control,attack,tool,...,have,an,next,year,dropbox,python,what,we,learn,from
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# split the data into two sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)

In [17]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [19]:
# calculate our prediction error

mse = sum((predictions - y_test)**2)/len(predictions)
print(mse)

2980.349514089688


In [28]:
avg = submissions.upvotes.mean()
sd = submissions.upvotes.std()
rmse = mse**0.5
print(avg, sd, rmse)

10.095357142857143 39.498832249429036 54.59257746333001
